<a href="https://colab.research.google.com/github/srikarraju/GridWorld/blob/main/Natural_Actor_critic_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import gym
from collections import deque
import numpy as np

In [ ]:
class value_net(nn.Module):
  def __init__(self,state_dim,hidden_dim):
    super(value_net,self).__init__()
    self.h = nn.Linear(state_dim,hidden_dim)
    self.out = nn.Linear(hidden_dim,1)

  def forward(self,x):
    x = self.h(x)
    x = nn.ReLU()(x)
    x = self.out(x)
    return x

In [ ]:
env = gym.make('CartPole-v0')

value_fn = value_net(4,8)
optimizer2 = torch.optim.Adam(value_fn.parameters())

weights_w = np.zeros(5,dtype=float)
weights_p = np.zeros(5,dtype=float)
print(weights_w)
print(weights_p)

returns = deque(maxlen=100)

[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]


In [ ]:
alpha_0, beta_0, gamma, epsilon = 0.1, 0.01, 0.95, 0.1
alpha_c, beta_c = 1000, 1000
t = 0
n_episode = 1
actions_list = []
avg_reward = 0

fischer_inv = 1.5*np.eye(5)

while n_episode <=5000:
  rewards,states,actions = [],[],[]
  state = env.reset()

  while True:
    t += 1
    state_action = np.zeros((2,5),dtype=float)
    for i in range(4):
      state_action[0][i] = state[i]
      state_action[1][i] = state[i]
    state_action[0][4] = 0
    state_action[1][4] = 1

    probs = np.dot(np.asarray(state_action),weights_p)
    probs -= probs.max()
    probs = np.exp(np.clip(probs/epsilon, -500, 500))
    probs /= probs.sum()

    probs2 = probs.cumsum()
    action = np.where(probs2 >= np.random.random())[0][0]
    #print(action)

    new_state, reward, done, info = env.step(action.item())


    value_curr = value_fn(torch.tensor(state).unsqueeze(0).float())
    value_next = value_fn(torch.tensor(state).unsqueeze(0).float())


    beta = (beta_0 * beta_c) / (beta_c + t)
    alpha = (alpha_0 * alpha_c) / (alpha_c + t**(2/3))

    avg_reward = (1 - gamma)*avg_reward + gamma * reward
    td_error = reward + value_next - value_curr -avg_reward
    td_error = td_error.detach()
    td_error = td_error.item()

    value_net_loss = -0.1*alpha* td_error*value_curr
    optimizer2.zero_grad()
    value_net_loss.backward()
    optimizer2.step()

    grad_prob = probs[action]*(state_action[action] - np.dot(np.transpose(state_action),probs))

    Gt_psi_dot = np.dot(fischer_inv, grad_prob)
    fischer_inv -= (0.001*alpha * np.outer(Gt_psi_dot,np.transpose(Gt_psi_dot)) ) / (1 - 0.001*alpha + 0.001*alpha * np.dot(grad_prob,Gt_psi_dot))
    fischer_inv /= (1 - 0.001*alpha)

    weights_w = (1-alpha)*weights_w + alpha*td_error*np.dot(fischer_inv, grad_prob)

    weights_p += beta * weights_w

    states.append(state)
    actions.append(action)
    rewards.append(reward)
    actions_list.append(action)

    state = new_state
    if done==True:
      break

  returns.append(np.sum(rewards))
  #print(np.sum(rewards))
  if n_episode%100==0:
    print("Episode: {:6d}\tAvg. Return: {:6.2f}".format(n_episode, np.mean(returns)))
  #reinforce_baseline_returns.append(np.mean(returns))
  n_episode += 1

env.close()

Episode:    100	Avg. Return:  21.76
Episode:    200	Avg. Return:  21.25
Episode:    300	Avg. Return:  21.08
Episode:    400	Avg. Return:  24.67
Episode:    500	Avg. Return:  21.81
Episode:    600	Avg. Return:  22.92
Episode:    700	Avg. Return:  22.27
Episode:    800	Avg. Return:  22.63
Episode:    900	Avg. Return:  22.81
Episode:   1000	Avg. Return:  22.02
Episode:   1100	Avg. Return:  21.51
Episode:   1200	Avg. Return:  22.79
Episode:   1300	Avg. Return:  23.21
Episode:   1400	Avg. Return:  22.35
Episode:   1500	Avg. Return:  21.05
Episode:   1600	Avg. Return:  23.23
Episode:   1700	Avg. Return:  20.99
Episode:   1800	Avg. Return:  21.09
Episode:   1900	Avg. Return:  22.66
Episode:   2000	Avg. Return:  22.48
Episode:   2100	Avg. Return:  22.15
Episode:   2200	Avg. Return:  22.54
Episode:   2300	Avg. Return:  22.01
Episode:   2400	Avg. Return:  21.62
Episode:   2500	Avg. Return:  23.16
Episode:   2600	Avg. Return:  23.79
Episode:   2700	Avg. Return:  22.50
Episode:   2800	Avg. Return:

KeyboardInterrupt: ignored

In [ ]:
env = gym.make('CartPole-v0')

weights_v = np.zeros(4,dtype=float)
weights_w = np.zeros(5,dtype=float)
weights_p = np.zeros(5,dtype=float)
print(weights_v)
print(weights_p)

returns = deque(maxlen=100)

[0. 0. 0. 0.]
[0. 0. 0. 0. 0.]


In [ ]:
alpha_0, beta_0, gamma, epsilon = 0.1, 0.01, 0.95, 0.1
alpha_c, beta_c = 1000, 1000
t = 0
n_episode = 1
actions_list = []
avg_reward = 0

fischer_inv = 1.5*np.eye(5)

while n_episode <=5000:
  rewards,states,actions = [],[],[]
  state = env.reset()

  state = np.asarray(state)
  value_curr = np.dot(weights_v,state)
  #print(value_curr)

  while True:
    t += 1
    state_action = np.zeros((2,5),dtype=float)
    for i in range(4):
      state_action[0][i] = state[i]
      state_action[1][i] = state[i]
    state_action[0][4] = 0
    state_action[1][4] = 1

    probs = np.dot(np.asarray(state_action),weights_p)
    probs -= probs.max()
    probs = np.exp(np.clip(probs/epsilon, -500, 500))
    probs /= probs.sum()

    probs2 = probs.cumsum()
    action = np.where(probs2 >= np.random.random())[0][0]
    #print(action)

    new_state, reward, done, info = env.step(action.item())


    value_curr = np.dot(weights_v,np.asarray(state))
    value_next = np.dot(weights_v,np.asarray(new_state))

    avg_reward = (1 - gamma)*avg_reward + gamma * reward

    td_error = reward + value_curr - value_next - avg_reward

    beta = (beta_0 * beta_c) / (beta_c + t)
    alpha = (alpha_0 * alpha_c) / (alpha_c + t**(2/3))

    weights_v += alpha * td_error * np.asarray(state)

    grad_prob = probs[action]*(state_action[action] - np.dot(np.transpose(state_action),probs))

    Gt_psi_dot = np.dot(fischer_inv, grad_prob)
    fischer_inv -= (alpha * np.outer(Gt_psi_dot,np.transpose(Gt_psi_dot)) ) / (1 - alpha + alpha * np.dot(grad_prob,Gt_psi_dot))
    fischer_inv /= (1 - alpha)

    weights_w = (1-alpha)*weights_w + alpha*td_error*np.dot(fischer_inv, grad_prob)

    weights_p += beta * weights_w

    states.append(state)
    actions.append(action)
    rewards.append(reward)
    actions_list.append(action)

    state = new_state
    if done==True:
      break

  returns.append(np.sum(rewards))
  #print(np.sum(rewards))
  if n_episode%100==0:
    print("Episode: {:6d}\tAvg. Return: {:6.2f}".format(n_episode, np.mean(returns)))
  #reinforce_baseline_returns.append(np.mean(returns))
  n_episode += 1

env.close()

Episode:    100	Avg. Return:  19.63
Episode:    200	Avg. Return:  21.22
Episode:    300	Avg. Return:  24.25
Episode:    400	Avg. Return:  24.35
Episode:    500	Avg. Return:  22.77
Episode:    600	Avg. Return:  22.29
Episode:    700	Avg. Return:  24.88
Episode:    800	Avg. Return:  22.90
Episode:    900	Avg. Return:  22.35
Episode:   1000	Avg. Return:  21.06
Episode:   1100	Avg. Return:  20.40
Episode:   1200	Avg. Return:  20.21
Episode:   1300	Avg. Return:  20.73
Episode:   1400	Avg. Return:  19.50
Episode:   1500	Avg. Return:  20.27
Episode:   1600	Avg. Return:  18.74
Episode:   1700	Avg. Return:  18.40
Episode:   1800	Avg. Return:  19.62
Episode:   1900	Avg. Return:  18.78
Episode:   2000	Avg. Return:  20.36
Episode:   2100	Avg. Return:  18.21
Episode:   2200	Avg. Return:  19.30
Episode:   2300	Avg. Return:  20.38
Episode:   2400	Avg. Return:  19.56
Episode:   2500	Avg. Return:  19.54
Episode:   2600	Avg. Return:  21.50
Episode:   2700	Avg. Return:  18.29
Episode:   2800	Avg. Return: